#Introducción
En este ejercicio se implento un algoritmo de busqueda utilizando el las capacidades que nos ofrece la programacion en pralelo para poder realizar las busquedas con un costo computacional constante.
Adicioanlmente se impleneto un sencillo algoritmo de busqueda secuencial para ejemplificar las diferencias en tiempo de ejecucion que pueden darce ante grandes volumenes de datos.

#Armado del ambiente

In [2]:
!pip install pycuda

     |████████████████████████████████| 1.7MB 4.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627098 sha256=29193e4cf9a1b250e0d31d34c5fcafc66d744c63c09242f360562a28905c14f6
  Stored in directory: /root/.cache/pip/wheels/d5/55/64/fd4dddcc5f1c25eebd90b5291c3769101dc978c70165685512
Successfully built pycuda
  Created wheel for pytools: filename=pytools-2021.2.7-py2.py3-none-any.whl size=60644 sha256=a5a68d1c5d5d38a21c02518624b0899643090df898e31505cd05bb48457a019e
  Stored in directory: /root/.cache/pip/wheels/a0/b5/e5/e65d25997fd77729b9aa214645add18688483e48bbcbab6ffc
Successfully built pytools


#Desarrollo

In [25]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   100000000#@param {type: "number"}
nro_a_buscar = 11#@param {type: "number"}


if not isinstance(cantidad_N,int) or cantidad_N < 1:
  raise TypeError("cantidad_N no es un número válido.")

if not isinstance(nro_a_buscar,int):
  raise TypeError("nro_a_buscar no es un número válido.")


# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


#Por la dificultad en encontrar un dataset con el bolumen de datos suficiente y la incapacidad de almacenarlo para
#su uso estamos generando a modo de ejemplo un sensillo array con numeros aleatorios en los que buscar un elemento.

aux= numpy.zeros((1,), dtype=int);
aux=aux.astype(numpy.int32())
x_cpu= numpy.random.randint(0, 1000, size = cantidad_N)

x_cpu = x_cpu.astype(numpy.int32())

#En CPU
tiempo_cpu = datetime.now()

valor_encontrado = False

for numero in x_cpu:
  if numero == nro_a_buscar:
    valor_encontrado = True


print( "Cantidad de elementos: ", len(x_cpu) )
print("Tiempo CPU: ", tiempo_en_ms( datetime.now() - tiempo_cpu   ), "[ms]" )

if valor_encontrado:
  print("Se encontró por CPU el valor:",nro_a_buscar)
else:
  print("No se encontró por CPU el valor:",nro_a_buscar)




aux2= numpy.empty_like(aux)

x_aux= cuda.mem_alloc(aux.nbytes)

cuda.memcpy_htod(x_aux, aux)

r_cpu = numpy.empty_like( x_cpu )

x_gpu = cuda.mem_alloc( x_cpu.nbytes )

cuda.memcpy_htod( x_gpu, x_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_axpy( int n, int *X, int nro_a_buscar, int *res )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    if (X[idx] == nro_a_buscar) {
     // X[0] = nro_a_buscar;
     res[0]=1;
    }
  }
}
""")
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_axpy")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (len(x_cpu)+dim_hilo-1) / dim_hilo )
print( "-----------------------------------------------")
print( "-----------------------------------------------")
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(len(x_cpu)), x_gpu, numpy.int32(nro_a_buscar), x_aux,block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu


# GPU - Copio el resultado desde la memoria GPU.


tiempo_total = datetime.now() - tiempo_total
cuda.memcpy_dtoh( r_cpu, x_gpu )
result= r_cpu.tolist()
cuda.memcpy_dtoh( aux2, x_aux )
result_2= aux2.tolist()

print( "Cantidad de elementos: ", len(x_cpu) )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

if result_2[0] == 1:
  print("Se encontró por GPU el valor:",nro_a_buscar)
else:
  print("No se encontró por GPU el valor:",nro_a_buscar)

print( "-----------------------------------------------")
print( "-----------------------------------------------")
print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )

Cantidad de elementos:  100000000
Tiempo CPU:  165620.041 [ms]
Se encontró por CPU el valor: 11
-----------------------------------------------
-----------------------------------------------
Thread x:  256 , Bloque x: 390625
Cantidad de elementos:  100000000
Thread x:  256 , Bloque x: 390625
Tiempo GPU:  0.749 [ms]
Se encontró por GPU el valor: 11
-----------------------------------------------
-----------------------------------------------
Tiempo TOTAL:  166493.404 [ms]


#Metricas
Al ejecutar el algorimo con 10 millones de elementos pordemos ver los siguientes resultados:
##CPU: 16929.319 [ms]
##GPU: 0.759 [ms]
Al ejecutar el algorimo con 100 mil de elementos pordemos ver los siguientes resultados:
##CPU:  170.655 [ms]
##GPU: 0.628 [ms]
Al ejecutar el algorimo con 1 de elementos pordemos ver los siguientes resultados:
##CPU:  170.655 [ms]
##GPU: 0.628 [ms]
Al ejecutar el algorimo con 1 de elementos pordemos ver los siguientes resultados:
##CPU:  1.177 [ms]
##GPU: 0.698 [ms]

#Conclusión
Comparando notamos que el algoritmo de busqueda secuencial por cpu cumple que su complejidad O(n) dando tiempo de ejecucion sumanente largo pasado cierto nuemro de elementos.
por otro lado el algoritmo de busqueda acelerado por GPU, con su complejidad O(1) manteniendo tiempos de busqueda en torno a <1ms sin impostar la cantidad de lementos del array
Si bien la implementacion es realizada es simple es muy falcil partir de ella para realizar un algoritmo mucho mas complejo que sea util para el analisis de datos en el mundo real

#Bibliografía
https://github.com/wvaliente/SOA_HPC/blob/main/Ejercicios/Prueba%201%20-%20Vectores%20-%20GPU.ipynb